<a href="https://colab.research.google.com/github/niikun/langchain_tutorial/blob/main/Vector_stores_and_retrievers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Vector stores and retrievers

##Setup

In [1]:
!pip install langchain langchain-chroma langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.5/308.5 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.8/122.8 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 44.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 52.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import os
from google.colab import userdata
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

## Documents
LangChainはDocument抽象化を実装しており、テキストの単位と関連するメタデータを表現することを目的としています。2つの属性を持ちます：

- page_content: コンテンツを表す文字列  
- metadata: 任意のメタデータを含むdict  

metadata属性は、文書のソースや他の文書との関係などの情報を取り込むことができます。個々のDocumentオブジェクトは、より大きな文書の塊を表すことが多いことに注意してください。

いくつかのサンプル・ドキュメントを生成してみましょう：

In [4]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata={"source": "fish-pets-doc"},
    ),
    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata={"source": "bird-pets-doc"},
    ),
    Document(
        page_content="Rabbits are social animals that need plenty of space to hop around.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

In [7]:
documents[0]

Document(page_content='Dogs are great companions, known for their loyalty and friendliness.', metadata={'source': 'mammal-pets-doc'})

## Vector stores
ベクトル検索は、構造化されていないデータ（構造化されていないテキストなど）を保存し、検索する一般的な方法である。  
このアイデアは、テキストに関連する数値ベクトルを格納することである。  
クエリが与えられたら、それを同じ次元のベクトルとして埋め込み、ベクトル類似度メトリクスを使ってストア内の関連データを特定します。  

LangChain VectorStoreオブジェクトには、テキストやDocumentオブジェクトをストアに追加したり、さまざまな類似度メトリクスを使ってクエリを実行したりするためのメソッドが含まれています。  
これらはしばしば埋め込みモデルで初期化され、テキストデータがどのように数値ベクトルに変換されるかを決定します。

LangChainは様々なベクターストア技術との統合スイートを含みます。  
いくつかのベクトルストアはプロバイダ（例：様々なクラウドプロバイダ）によってホストされ、使用するには特定の認証情報が必要です。  
また、いくつかの（Postgresのような）ベクトルストアは、ローカルまたはサードパーティ経由で実行できる別のインフラで実行されます。  
ここでは、インメモリ実装を含む**Chroma**を使ってLangChain VectorStoresの使い方をデモします。

ベクターストアをインスタンス化するには、テキストをどのように数値ベクターに変換するかを指定するエンベッディングモデルを提供する必要があります。  
ここではOpenAIのエンベッディングを使います。

In [12]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=OpenAIEmbeddings()
)

ここで .from_documents を呼び出すと、ドキュメントがベクターストアに追加されます。  
VectorStoreはドキュメントを追加するメソッドを実装しており、オブジェクトがインスタンス化された後に呼び出すこともできます。  
ほとんどの実装では、既存のベクターストアに接続することができます -- 例えば、クライアントやインデックス名、その他の情報を提供することで。  
詳しくは各インテグレーションのドキュメントを参照してください。

ドキュメントを含むVectorStoreをインスタンス化したら、それをクエリできます。VectorStoreにはクエリ用のメソッドがあります：
- 同期および非同期  
- 文字列クエリーとベクトル
- 類似性スコアを返す場合と返さない場合
- 類似度と最大限界関連性（検索結果の多様性とクエリの類似度のバランスをとる）。

In [32]:
vectorstore

In [14]:
vectorstore.similarity_search("cat")

[Document(page_content='Cats are independent pets that often enjoy their own space.', metadata={'source': 'mammal-pets-doc'}),
 Document(page_content='Cats are independent pets that often enjoy their own space.', metadata={'source': 'mammal-pets-doc'}),
 Document(page_content='Dogs are great companions, known for their loyalty and friendliness.', metadata={'source': 'mammal-pets-doc'}),
 Document(page_content='Dogs are great companions, known for their loyalty and friendliness.', metadata={'source': 'mammal-pets-doc'})]

In [18]:
vectorstore.similarity_search_with_score("cat")

[(Document(page_content='Cats are independent pets that often enjoy their own space.', metadata={'source': 'mammal-pets-doc'}),
  0.37532690167427063),
 (Document(page_content='Cats are independent pets that often enjoy their own space.', metadata={'source': 'mammal-pets-doc'}),
  0.37532690167427063),
 (Document(page_content='Dogs are great companions, known for their loyalty and friendliness.', metadata={'source': 'mammal-pets-doc'}),
  0.4833090305328369),
 (Document(page_content='Dogs are great companions, known for their loyalty and friendliness.', metadata={'source': 'mammal-pets-doc'}),
  0.4833090305328369)]

埋め込まれたクエリとの類似性に基づいて文書を返す：

In [28]:
embedding = OpenAIEmbeddings().embed_query("cat")
print(len(embedding))
embedding[:10]

1536


[-0.007064840399927003,
 -0.017335813760570657,
 -0.009703516178850385,
 -0.03069942864557231,
 -0.012505334995320574,
 0.003071361501690948,
 -0.00510711370481684,
 -0.04122575808726891,
 -0.014612019287962963,
 -0.021308012636139663]

In [22]:
vectorstore.similarity_search_by_vector(embedding)

[Document(page_content='Cats are independent pets that often enjoy their own space.', metadata={'source': 'mammal-pets-doc'}),
 Document(page_content='Cats are independent pets that often enjoy their own space.', metadata={'source': 'mammal-pets-doc'}),
 Document(page_content='Dogs are great companions, known for their loyalty and friendliness.', metadata={'source': 'mammal-pets-doc'}),
 Document(page_content='Dogs are great companions, known for their loyalty and friendliness.', metadata={'source': 'mammal-pets-doc'})]

## Retrievers
LangChain VectorStoreオブジェクトはRunnableのサブクラスではないので、すぐにLangChain Expression Languageチェーンに統合することはできません。

**LangChain RetrieverはRunnableなので、標準的なメソッド・セット（同期・非同期呼び出しやバッチ操作など）を実装し、LCELチェーンに組み込めるように設計されています。**

Retrieverをサブクラス化することなく、これの簡単なバージョンを自分で作ることができる。どのようなメソッドを使って文書を取得したいかを決めれば、簡単に実行可能なものを作ることができる。以下では、similarity_searchメソッドを中心に作成する：

### RunnableLambdaとは？
RunnableLambdaは、任意のPython関数をラップして、LangChainのパイプラインで実行可能にするためのクラスです。これにより、VectorStoreをパイプラインの一部として簡単に組み込むことができます。

In [35]:
from typing import List

from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

retriever = RunnableLambda(vectorstore.similarity_search).bind(k=1) #select top result

retriever.batch(["cat","shark"])

[[Document(page_content='Cats are independent pets that often enjoy their own space.', metadata={'source': 'mammal-pets-doc'})],
 [Document(page_content='Goldfish are popular pets for beginners, requiring relatively simple care.', metadata={'source': 'fish-pets-doc'})]]

VectorStoreRetrieverは "similarity"(デフォルト)、"mmr"(最大限界関連性、前述)、"similarity_score_threshold "の検索タイプをサポートしています。後者を使うことで、リトリーバが出力するドキュメントを類似度スコアで閾値付けすることができる。

リトリーバーは、与えられた質問と検索されたコンテキストを組み合わせて、LLMのプロンプトを生成するRAG(retrieval-augmented generation)アプリケーションなど、より複雑なアプリケーションに簡単に組み込むことができる。以下に最小限の例を示す。

### Openai

In [37]:
!pip install -qU langchain-openai

In [38]:
from langchain_openai import OpenAI

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o")

runnable

In [41]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message="""
Answer the following question using the context only in japanese.

{question}

Context:
{context}
"""
prompt = ChatPromptTemplate.from_messages(["human",message])
rag_chain = {"context":retriever,"question":RunnablePassthrough()} | prompt |llm

response = rag_chain.invoke("tell me about cats")

In [42]:
print(response)

content='猫は独立したペットであり、しばしば自分の空間を楽しむことが多いです。' response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 60, 'total_tokens': 88}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_3196d36131', 'finish_reason': 'stop', 'logprobs': None} id='run-f272501a-76f0-4bb4-af5c-1474b74dbd2d-0'


### {"context":retriever,"question":RunnablePassthrough()}の意味
この構文は、複数の入力を一つのパイプラインにまとめる方法を示しています。具体的には、contextとしてretrieverを使用し、questionとしてRunnablePassthrough()を使用しています。

context: これは、文脈情報を提供するための入力です。retrieverは、文脈情報を取得するためのメカニズム（例えば、検索エンジンやデータベースクエリ）です。
question: これは、ユーザーからの質問をそのままパイプラインに渡すためのものです。RunnablePassthrough()は、入力をそのまま出力に渡すためのクラスです。
この方法は、パイプラインに複数の異なるソースからのデータを入力するために使用されます。

### |ではなく、辞書を使う理由
|を使ってパイプラインを構築する方法は、単一のデータフローに適しています。しかし、複数の異なる入力（例えば、文脈と質問の2つの異なる入力）を同時に処理する必要がある場合は、辞書を使って入力をマッピングする方法が適しています。

In [43]:
{"context":retriever,"question":RunnablePassthrough()}

{'context': RunnableBinding(bound=RunnableLambda(similarity_search), kwargs={'k': 1}),
 'question': RunnablePassthrough()}